In [1]:
import csv
import time
import random
import jsonlines
import pandas as pd
import math

In [2]:
def question11_gen(data, num_questions_per_template, questions, question_id):
    print(len(data))
    new_data = data.loc[data["Cancelled"]==True]
    print(len(new_data))
    candidate_list = []
    for i in range(len(new_data)):
        candidate_list.append([new_data.iloc[i]["FlightDate"], new_data.iloc[i]["Origin"]])
    print(len(candidate_list))
    flight_date = None
    origin = None
    row_idx = 0
    while question_id < num_questions_per_template:
        random_indices = random.randint(0, len(candidate_list)-1)
        flightdate = candidate_list[random_indices][0]
        origin = candidate_list[random_indices][1]
        num_total = len(data.loc[(data["FlightDate"] == flightdate) & (data["Origin"] == origin)])
        num_cancelled = len(data.loc[(new_data["FlightDate"] == flightdate) & (data["Origin"] == origin) & (data["Cancelled"] == True)])
        if num_cancelled > 0:
            question = "What percentage of the flights from {} were delayed on {}?".format(origin, flightdate)
            answer = "{:.1f}".format(num_cancelled / num_total * 100)+"%"
            questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
            question_id += 1
    return questions, question_id

In [4]:
def question12_gen(data, num_questions_per_template, questions, question_id):
    origin_list = data["Origin"].unique()
    print(len(origin_list))
    flightdate_list = data["FlightDate"].unique()
    print(len(flightdate_list))
    target = question_id + num_questions_per_template
    while question_id < target:
        origin_idx = random.randint(0, len(origin_list)-1)
        flightdate_idx = random.randint(0, len(flightdate_list)-1)
        origin = origin_list[origin_idx]
        flightdate = flightdate_list[flightdate_idx]
        sub_table = data.loc[(data["Origin"] == origin) & (data["FlightDate"] == flightdate)]
        total_delay = 0
        num_delay = 0
        for row_id in range(len(sub_table)):
            if not math.isnan(sub_table.iloc[row_id]["DepDelayMinutes"]):
                num_delay += 1
                total_delay += float(sub_table.iloc[row_id]["DepDelayMinutes"])
        if num_delay != 0:
            avg_delay = total_delay / len(sub_table)
            question = "What is the average delay time of all the flights that departed from {} on {}?".format(origin, flightdate)
            answer = "{:.1f}".format(avg_delay)
            questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
            question_id += 1
    return questions, question_id

In [5]:
def question13_gen(data, num_questions_per_template, questions, question_id):
    flightdate_list = data["FlightDate"].unique()
    # for date in flightdate_list:
    target = question_id + num_questions_per_template
    while question_id < target:
        date_idx = random.randint(0, len(flightdate_list)-1)
        date = flightdate_list[date_idx]
        num_diverted = len(data.loc[(data["FlightDate"] == date) & (data["Diverted"] == True)])
        num_total = len(data.loc[(data["FlightDate"] == date)])
        question = "How many flights were diverted on {}?".format(date)
        answer = str(num_diverted)
        questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
        question_id += 1
    return questions, question_id

In [6]:
def question14_gen(data, num_questions_per_template, questions, question_id):
    flightdate_list = data["FlightDate"].unique()
    # for date in flightdate_list:
    target = question_id + num_questions_per_template
    while question_id < target:
        date_idx = random.randint(0, len(flightdate_list)-1)
        date = flightdate_list[date_idx]
        num = len(data.loc[(data["FlightDate"] == date) & (data["Distance"] > 300.0)])
        question = "How many flights with a distance greater than 500 miles on {}?".format(date)
        answer = str(num)
        questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
        question_id += 1
    return questions, question_id

In [7]:
def question15_gen(data, num_questions_per_template, questions, question_id):
    # What is the average airtime of the flights from {origin} to {dest} host by {airline}?
    airline_list = data["Airline"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        airline_idx = random.randint(0, len(airline_list)-1)
        airline = airline_list[airline_idx]
        sub_table = data.loc[(data["Airline"] == airline)]

        origin_list = sub_table["Origin"].unique()
        origin_idx = random.randint(0, len(origin_list)-1)
        origin = origin_list[origin_idx]
        sub_table = sub_table.loc[(sub_table["Origin"] == origin)]

        dest_list = sub_table["Dest"].unique()
        dest_idx = random.randint(0, len(dest_list)-1)
        dest = dest_list[dest_idx]
        sub_table = sub_table.loc[(sub_table["Dest"] == dest)]
        if len(sub_table) != 0:
            sum_airtime = 0
            num_airtime = 0
            for row_id in range(len(sub_table)):
                if not math.isnan(sub_table.iloc[row_id]["AirTime"]):
                    sum_airtime += sub_table.iloc[row_id]["AirTime"]
                    num_airtime += 1
            if num_airtime != 0:
                avg_airtime = sum_airtime / num_airtime
                question = "What is the average airtime of the flights from {} to {} host by {}?".format(origin, dest, airline)
                answer = "{:.1f} minutes".format(avg_airtime)
                questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
                question_id += 1
    return questions, question_id

In [8]:
def question16_gen(data, num_questions_per_template, questions, question_id):
    airline_list = data["Airline"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        airline_idx = random.randint(0, len(airline_list)-1)
        airline = airline_list[airline_idx]
        sub_table = data.loc[(data["Airline"] == airline)]

        origin_list = sub_table["Origin"].unique()
        origin_idx = random.randint(0, len(origin_list)-1)
        origin = origin_list[origin_idx]
        sub_table = sub_table.loc[(sub_table["Origin"] == origin)]

        dest_list = sub_table["Dest"].unique()
        dest_idx = random.randint(0, len(dest_list)-1)
        dest = dest_list[dest_idx]
        sub_table = sub_table.loc[(sub_table["Dest"] == dest)]
        if len(sub_table) != 0:
            num_airlines = len(sub_table["Flight_Number_Marketing_Airline"].unique())
            question = "How many flights from {} to {} host by {}?".format(origin, dest, airline)
            answer = num_airlines
            questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
            question_id += 1
    return questions, question_id

In [9]:
def question17_gen(data, num_questions_per_template, questions, question_id):
    number_list = data["Flight_Number_Marketing_Airline"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        number_idx = random.randint(0, len(number_list)-1)
        number = number_list[number_idx]
        sub_table = data.loc[(data["Flight_Number_Marketing_Airline"] == number)]
        carrier_list = sub_table["IATA_Code_Marketing_Airline"].unique()
        carrier_idx = random.randint(0, len(carrier_list)-1)
        carrier = carrier_list[carrier_idx]
        sub_table = sub_table.loc[(sub_table["IATA_Code_Marketing_Airline"] == carrier)]
        if len(sub_table) != 0:
            avg_airtime = 0
            num_airtime = 0
            for row_id in range(len(sub_table)):
                if not math.isnan(sub_table.iloc[row_id]["AirTime"]):
                    avg_airtime += sub_table.iloc[row_id]["AirTime"]
                    num_airtime += 1
            if num_airtime != 0:
                avg_airtime = avg_airtime / num_airtime
                question = "What is the average flight time of {}{}?".format(carrier, number)
                answer = "{:.1f} minutes".format(avg_airtime)
                questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
                question_id += 1
    return questions, question_id

In [10]:
def question18_gen(data, num_questions_per_template, questions, question_id):
    # What is the fastest flight from {origin} to {dest} on {DayofWeek}?
    origin_list = data["Origin"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        origin_idx = random.randint(0, len(origin_list)-1)
        origin = origin_list[origin_idx]
        sub_table = data.loc[(data["Origin"] == origin)]

        dest_list = sub_table["Dest"].unique()
        dest_idx = random.randint(0, len(dest_list)-1)
        dest = dest_list[dest_idx]
        sub_table = sub_table.loc[(sub_table["Dest"] == dest)]

        dayofweek_list = sub_table["DayOfWeek"].unique()
        dayofweek_idx = random.randint(0, len(dayofweek_list)-1)
        dayofweek = dayofweek_list[dayofweek_idx]
        sub_table = sub_table.loc[(sub_table["DayOfWeek"] == dayofweek)]
        if len(sub_table) != 0:
            min_airtime = 100000
            for row_id in range(len(sub_table)):
                if sub_table.iloc[row_id]["AirTime"] < min_airtime:
                    min_airtime = sub_table.iloc[row_id]["AirTime"]
                    min_carrier = sub_table.iloc[row_id]["IATA_Code_Marketing_Airline"]
                    min_number = sub_table.iloc[row_id]["Flight_Number_Marketing_Airline"]
            question = "What is the fastest flight from {} to {} on {}?".format(origin, dest, dayofweek)
            answer = "{}{}".format(min_carrier, min_number)
            questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
            question_id += 1
    return questions, question_id

In [11]:
def question19_gen(data, num_questions_per_template, questions, question_id):
    # What is the average speed of {carrier}{number} from {origin} to {dest}?
    origin_list = data["Origin"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        origin_idx = random.randint(0, len(origin_list)-1)
        origin = origin_list[origin_idx]
        sub_table = data.loc[(data["Origin"] == origin)]

        dest_list = sub_table["Dest"].unique()
        dest_idx = random.randint(0, len(dest_list)-1)
        dest = dest_list[dest_idx]
        sub_table = sub_table.loc[(sub_table["Dest"] == dest)]

        # if len(sub_table) != 0:
        number_list = sub_table["Flight_Number_Marketing_Airline"].unique()
        number_idx = random.randint(0, len(number_list)-1)
        number = number_list[number_idx]
        sub_table = sub_table.loc[(sub_table["Flight_Number_Marketing_Airline"] == number)]
        carrier_list = sub_table["IATA_Code_Marketing_Airline"].unique()
        carrier_idx = random.randint(0, len(carrier_list)-1)
        carrier = carrier_list[carrier_idx]
        sub_table = sub_table.loc[(sub_table["IATA_Code_Marketing_Airline"] == carrier)]
        if len(sub_table) != 0:
            question = "What is the average speed of {}{} from {} to {}?".format(carrier, number, origin, dest)
            avg_speed = 0
            num_speed = 0
            for row_id in range(len(sub_table)):
                if not math.isnan(sub_table.iloc[row_id]["AirTime"]):
                    avg_speed += sub_table.iloc[row_id]["Distance"] / sub_table.iloc[row_id]["AirTime"]
                    num_speed += 1
            if num_speed != 0:
                avg_speed = avg_speed / num_speed
                answer = "{:.1f} miles per minute".format(avg_speed)
                questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
                question_id += 1
    return questions, question_id

In [12]:
def question20_gen(data, num_questions_per_template, questions, question_id):
    # What is the total number of flights operated by {airline} on {date}?
    airline_list = data["Airline"].unique()
    target = question_id + num_questions_per_template
    while question_id < target:
        airline_idx = random.randint(0, len(airline_list)-1)
        airline = airline_list[airline_idx]
        sub_table = data.loc[(data["Airline"] == airline)]

        flightdate_list = sub_table["FlightDate"].unique()
        flightdate_idx = random.randint(0, len(flightdate_list)-1)
        flightdate = flightdate_list[flightdate_idx]
        sub_table = sub_table.loc[(sub_table["FlightDate"] == flightdate)]
        if len(sub_table) != 0:
            question = "What is the total number of flights operated by {} on {}?".format(airline, flightdate)
            answer = str(len(sub_table))
            questions.append({"qid": "hard-flight-{:0>4d}".format(question_id), "question":question, "answer":answer})
            question_id += 1
    return questions, question_id

In [13]:
start_time = time.time()
file_path = "/<YOUR_OWN_PATH>/ToolQA/data/raw_data/flights/Combined_Flights_2022.csv"
data = pd.read_csv(file_path)
num_questions_per_template = 10
question_id = 0
questions = []

In [14]:
# question template 11
questions, question_id = question11_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 11: {} seconds".format(end_time - start_time))

4078318
123192
123192
[{'qid': 'hard-flight-0000', 'question': 'What percentage of the flights from PIT were delayed on 2022-05-17?', 'answer': '1.7%'}, {'qid': 'hard-flight-0001', 'question': 'What percentage of the flights from JAC were delayed on 2022-02-24?', 'answer': '11.1%'}, {'qid': 'hard-flight-0002', 'question': 'What percentage of the flights from DEN were delayed on 2022-02-02?', 'answer': '15.0%'}, {'qid': 'hard-flight-0003', 'question': 'What percentage of the flights from HPN were delayed on 2022-01-30?', 'answer': '37.1%'}, {'qid': 'hard-flight-0004', 'question': 'What percentage of the flights from STT were delayed on 2022-03-19?', 'answer': '4.0%'}, {'qid': 'hard-flight-0005', 'question': 'What percentage of the flights from ORD were delayed on 2022-06-13?', 'answer': '6.9%'}, {'qid': 'hard-flight-0006', 'question': 'What percentage of the flights from LAW were delayed on 2022-04-24?', 'answer': '50.0%'}, {'qid': 'hard-flight-0007', 'question': 'What percentage of the

In [15]:
# question template 12
questions, question_id = question12_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 12: {} seconds".format(end_time - start_time))

375
212
[{'qid': 'hard-flight-0010', 'question': 'What is the average delay time of all the flights that departed from ERI on 2022-01-18?', 'answer': '23.5'}, {'qid': 'hard-flight-0011', 'question': 'What is the average delay time of all the flights that departed from PBG on 2022-07-31?', 'answer': '0.0'}, {'qid': 'hard-flight-0012', 'question': 'What is the average delay time of all the flights that departed from TBN on 2022-05-23?', 'answer': '0.0'}, {'qid': 'hard-flight-0013', 'question': 'What is the average delay time of all the flights that departed from SUX on 2022-07-12?', 'answer': '0.0'}, {'qid': 'hard-flight-0014', 'question': 'What is the average delay time of all the flights that departed from BRW on 2022-03-09?', 'answer': '19.0'}, {'qid': 'hard-flight-0015', 'question': 'What is the average delay time of all the flights that departed from PSG on 2022-02-06?', 'answer': '0.0'}, {'qid': 'hard-flight-0016', 'question': 'What is the average delay time of all the flights that

In [16]:
# question template 13
questions, question_id = question13_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 13: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0020', 'question': 'How many flights were diverted on 2022-01-14?', 'answer': '34'}, {'qid': 'hard-flight-0021', 'question': 'How many flights were diverted on 2022-01-05?', 'answer': '30'}, {'qid': 'hard-flight-0022', 'question': 'How many flights were diverted on 2022-05-11?', 'answer': '33'}, {'qid': 'hard-flight-0023', 'question': 'How many flights were diverted on 2022-04-23?', 'answer': '31'}, {'qid': 'hard-flight-0024', 'question': 'How many flights were diverted on 2022-01-19?', 'answer': '54'}, {'qid': 'hard-flight-0025', 'question': 'How many flights were diverted on 2022-04-18?', 'answer': '26'}, {'qid': 'hard-flight-0026', 'question': 'How many flights were diverted on 2022-03-01?', 'answer': '16'}, {'qid': 'hard-flight-0027', 'question': 'How many flights were diverted on 2022-04-02?', 'answer': '41'}, {'qid': 'hard-flight-0028', 'question': 'How many flights were diverted on 2022-07-10?', 'answer': '69'}, {'qid': 'hard-flight-0029', 'question': 'How 

In [17]:
# question template 14
questions, question_id = question14_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 14: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0030', 'question': 'How many flights with a distance greater than 500 miles on 2022-01-02?', 'answer': '16588'}, {'qid': 'hard-flight-0031', 'question': 'How many flights with a distance greater than 500 miles on 2022-01-09?', 'answer': '15583'}, {'qid': 'hard-flight-0032', 'question': 'How many flights with a distance greater than 500 miles on 2022-03-18?', 'answer': '16556'}, {'qid': 'hard-flight-0033', 'question': 'How many flights with a distance greater than 500 miles on 2022-02-25?', 'answer': '16354'}, {'qid': 'hard-flight-0034', 'question': 'How many flights with a distance greater than 500 miles on 2022-04-15?', 'answer': '16515'}, {'qid': 'hard-flight-0035', 'question': 'How many flights with a distance greater than 500 miles on 2022-03-15?', 'answer': '14833'}, {'qid': 'hard-flight-0036', 'question': 'How many flights with a distance greater than 500 miles on 2022-01-03?', 'answer': '16685'}, {'qid': 'hard-flight-0037', 'question': 'How many flights wit

In [18]:
# question template 15
questions, question_id = question15_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 15: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0040', 'question': 'What is the average airtime of the flights from BZN to CLT host by American Airlines Inc.?', 'answer': '198.2 minutes'}, {'qid': 'hard-flight-0041', 'question': 'What is the average airtime of the flights from MKE to CLT host by Comair Inc.?', 'answer': '95.2 minutes'}, {'qid': 'hard-flight-0042', 'question': 'What is the average airtime of the flights from ITO to OGG host by Hawaiian Airlines Inc.?', 'answer': '24.9 minutes'}, {'qid': 'hard-flight-0043', 'question': 'What is the average airtime of the flights from CLT to MOB host by Capital Cargo International?', 'answer': '85.6 minutes'}, {'qid': 'hard-flight-0044', 'question': 'What is the average airtime of the flights from CHO to CLT host by Capital Cargo International?', 'answer': '48.3 minutes'}, {'qid': 'hard-flight-0045', 'question': 'What is the average airtime of the flights from AMA to DFW host by Mesa Airlines Inc.?', 'answer': '49.3 minutes'}, {'qid': 'hard-flight-0046', 'question

In [19]:
# question template 16
questions, question_id = question16_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 16: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0050', 'question': 'How many flights from SLC to EWR host by United Air Lines Inc.?', 'answer': 7}, {'qid': 'hard-flight-0051', 'question': 'How many flights from JFK to IND host by Endeavor Air Inc.?', 'answer': 5}, {'qid': 'hard-flight-0052', 'question': 'How many flights from MSO to DEN host by Frontier Airlines Inc.?', 'answer': 1}, {'qid': 'hard-flight-0053', 'question': 'How many flights from ATL to CLE host by Spirit Air Lines?', 'answer': 1}, {'qid': 'hard-flight-0054', 'question': 'How many flights from ATW to ORD host by Air Wisconsin Airlines Corp?', 'answer': 13}, {'qid': 'hard-flight-0055', 'question': 'How many flights from FNT to ORD host by GoJet Airlines, LLC d/b/a United Express?', 'answer': 3}, {'qid': 'hard-flight-0056', 'question': 'How many flights from AVL to CLT host by Comair Inc.?', 'answer': 19}, {'qid': 'hard-flight-0057', 'question': 'How many flights from CLE to ORD host by Air Wisconsin Airlines Corp?', 'answer': 3}, {'qid': 'hard-fl

In [20]:
# question template 17
questions, question_id = question17_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 17: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0060', 'question': 'What is the average flight time of G41015?', 'answer': '192.5 minutes'}, {'qid': 'hard-flight-0061', 'question': 'What is the average flight time of NK2062?', 'answer': '119.1 minutes'}, {'qid': 'hard-flight-0062', 'question': 'What is the average flight time of AA5297?', 'answer': '73.0 minutes'}, {'qid': 'hard-flight-0063', 'question': 'What is the average flight time of G4886?', 'answer': '144.0 minutes'}, {'qid': 'hard-flight-0064', 'question': 'What is the average flight time of G41931?', 'answer': '98.6 minutes'}, {'qid': 'hard-flight-0065', 'question': 'What is the average flight time of AA3262?', 'answer': '46.7 minutes'}, {'qid': 'hard-flight-0066', 'question': 'What is the average flight time of WN6267?', 'answer': '136.2 minutes'}, {'qid': 'hard-flight-0067', 'question': 'What is the average flight time of WN6679?', 'answer': '147.0 minutes'}, {'qid': 'hard-flight-0068', 'question': 'What is the average flight time of DL1271?', 'answ

In [21]:
# question template 18
questions, question_id = question18_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 18: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0070', 'question': 'What is the fastest flight from BFL to DEN on 6?', 'answer': 'UA5839'}, {'qid': 'hard-flight-0071', 'question': 'What is the fastest flight from DVL to JMS on 2?', 'answer': 'UA5045'}, {'qid': 'hard-flight-0072', 'question': 'What is the fastest flight from SLC to BIL on 1?', 'answer': 'DL2178'}, {'qid': 'hard-flight-0073', 'question': 'What is the fastest flight from PIA to FLL on 4?', 'answer': 'G4899'}, {'qid': 'hard-flight-0074', 'question': 'What is the fastest flight from AVL to PIE on 6?', 'answer': 'G4504'}, {'qid': 'hard-flight-0075', 'question': 'What is the fastest flight from BIH to SFO on 3?', 'answer': 'UA4639'}, {'qid': 'hard-flight-0076', 'question': 'What is the fastest flight from ERI to CLT on 4?', 'answer': 'AA6096'}, {'qid': 'hard-flight-0077', 'question': 'What is the fastest flight from AKN to ANC on 3?', 'answer': 'AS2134'}, {'qid': 'hard-flight-0078', 'question': 'What is the fastest flight from DTW to CWA on 4?', 'answ

In [22]:
# question template 19
questions, question_id = question19_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 19: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0080', 'question': 'What is the average speed of DL4130 from HPN to DTW?', 'answer': '6.1 miles per minute'}, {'qid': 'hard-flight-0081', 'question': 'What is the average speed of WN1507 from COS to MDW?', 'answer': '7.9 miles per minute'}, {'qid': 'hard-flight-0082', 'question': 'What is the average speed of AA5272 from GPT to CLT?', 'answer': '7.4 miles per minute'}, {'qid': 'hard-flight-0083', 'question': 'What is the average speed of AA4086 from BPT to DFW?', 'answer': '5.3 miles per minute'}, {'qid': 'hard-flight-0084', 'question': 'What is the average speed of UA6124 from HSV to ORD?', 'answer': '5.7 miles per minute'}, {'qid': 'hard-flight-0085', 'question': 'What is the average speed of DL4239 from LSE to MSP?', 'answer': '4.1 miles per minute'}, {'qid': 'hard-flight-0086', 'question': 'What is the average speed of AA5165 from TRI to CLT?', 'answer': '4.0 miles per minute'}, {'qid': 'hard-flight-0087', 'question': 'What is the average speed of DL2993 from 

In [23]:
# question template 20
questions, question_id = question20_gen(data, num_questions_per_template, questions, question_id)
end_time = time.time()
print(questions[-10:])
print("Time elapsed for Question 20: {} seconds".format(end_time - start_time))

[{'qid': 'hard-flight-0090', 'question': 'What is the total number of flights operated by GoJet Airlines, LLC d/b/a United Express on 2022-02-01?', 'answer': '177'}, {'qid': 'hard-flight-0091', 'question': 'What is the total number of flights operated by United Air Lines Inc. on 2022-05-27?', 'answer': '1765'}, {'qid': 'hard-flight-0092', 'question': 'What is the total number of flights operated by Commutair Aka Champlain Enterprises, Inc. on 2022-03-24?', 'answer': '195'}, {'qid': 'hard-flight-0093', 'question': 'What is the total number of flights operated by Hawaiian Airlines Inc. on 2022-01-11?', 'answer': '190'}, {'qid': 'hard-flight-0094', 'question': 'What is the total number of flights operated by Hawaiian Airlines Inc. on 2022-06-04?', 'answer': '201'}, {'qid': 'hard-flight-0095', 'question': 'What is the total number of flights operated by Allegiant Air on 2022-04-12?', 'answer': '90'}, {'qid': 'hard-flight-0096', 'question': 'What is the total number of flights operated by F

In [24]:
with jsonlines.open('/<YOUR_OWN_PATH>/ToolQA/data/questions/hard/flight-hard.jsonl', mode='w') as writer:
    for row in questions:
        writer.write(row)